In [25]:
import json
import datetime
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib

Using matplotlib backend: Qt5Agg


In [26]:
file1=open('playerStats.json')
file2=open('teamsData.json')
playerStats=json.load(file1)
teamsData=json.load(file2)

# nationality analysis
---
- excat same analysis can be done for `bowlingStyle`[multiple category] 
- or for `right/left` handbad,bowl [binary]
- a bit diff analysis for age is done below

In [27]:
def getNations(playerStats):
    nationList=[]
    for player in playerStats:
        nation=player['player']['nationality']
        if not (nation in nationList):
            nationList.append(nation)
    return nationList

nations=getNations(playerStats)
# print(nations)


In [28]:
# in these fucntions other info lists also can be added like APIs
def getTeamNations(id,playerStats=playerStats,nationNames=nations):
    #set up dictoinary
    teamNations={'teamID':id}
    for nation in nationNames:
        teamNations.setdefault(nation,0)
    #add an extra key
    teamNations.setdefault('Foreigner',0)

    for player in playerStats:
        if player['teamID']==id:
            nation=player['player']['nationality']
            if nation=='Indian':
                teamNations[nation]+=1
            else:
                teamNations['Foreigner']+=1
                teamNations[nation]+=1
    
    return teamNations


In [29]:
nationalityList=[]
for team in teamsData:
    id=team['id']
    data=getTeamNations(id)
    data['Team Name']=team['abbreviation']
    nationalityList.append(data)

nationalityDF=pd.DataFrame(nationalityList)
nationalityDF.to_csv('nationlityOfTeams.csv',index=False)

In [30]:
totalOverall=list(nationalityDF.iloc[:,1:-1].sum(axis=0))
nationLabels=list(nationalityDF.columns)[1:-1]
overallDf=pd.DataFrame({'Nation':nationLabels,'Player Count':totalOverall})
overallDf.sort_values('Player Count',axis=0,inplace=True)

In [31]:
plt.barh(y=overallDf.Nation,width=overallDf['Player Count'],color='#FDB913')
plt.ylabel('Nationalities')
plt.xlabel('player count of particular nation')
plt.tight_layout()
plt.savefig('plots/nationality.png')
plt.show()

In [43]:
ageData =list()

for player in playerStats:
    data=player['player']
    birthday=data['dateOfBirth']
    #convert into date time object
    birthday=(datetime.datetime.strptime(birthday,'%Y-%m-%d').date())
    today=datetime.date.today()
    daysOld=(today-birthday).days
    daysInYear=365.25
    age=int(daysOld//daysInYear)

    ageData.append({'id':data['id'],'teamID':player['teamID'],'name':data['fullName'],'age':age})

In [56]:
ageDF=pd.DataFrame(ageData)
ageDF.sort_values('id',axis=0,inplace=True)

In [58]:
# basic sumarry of age data
ageDF.age.describe()

count    217.000000
mean      28.322581
std        4.904372
min       19.000000
25%       24.000000
50%       28.000000
75%       32.000000
max       42.000000
Name: age, dtype: float64

In [71]:
teamNames=[team['abbreviation'] for team in teamsData]
ageMeans=list(ageDF[['age','teamID']].groupby('teamID').mean()['age'])
plt.barh(y=teamNames,width=ageMeans,color='orange')
plt.xlabel('team avg age')
plt.ylabel('team abbr')
plt.tight_layout()
plt.savefig('plots/age.png')
plt.show()